<span type="title">异常处理</span> | <span type="update">2018-07-26</span> | <span type="version">1.0</span>
<span type="intro"><p class="card-text">本章主要讲解Java的异常处理机制。第一部分概要性质的讲解了异常的使用。首先讲解了（1）异常的基本概念，其中包括异常的类型定义、声明、抛出、监控、处理程序、处理模式。（2）自定义异常，其中包括异常的日志记录、详细信息记录。异常信息的获取，其中包括 Exception 各种方法的使用细节。（3）多个异常问题，包括：异常轨迹的获取、异常层次的处理。</p><p class="card-text">第二部分主要讲解了在标准中声明异常的限制问题、异常的执行顺序。之后介绍了异常的一些细节，比如构造器中的异常处理、继续声明异常还是完全处理异常、finally 语句中的使用细节等。最后的最后介绍了真实编程中的异常使用：不要过早的定义异常，也不要完全忽视异常。RuntimeException 是个好帮手。</p></span>

Java 对于错误或者说能够检测出结构不佳的代码，拒绝编译和运行。这一点对于异常处理尤甚。

# 异常基本概念

异常处理指的是，在出错的地方跳转到异常处理的地方，然后执行异常处理程序的代码。一般来说，通过在方法体中抛出一个异常可以导致Java异常处理流程。

**异常声明和抛出异常**

抛出异常需要在一个静态或者动态方法体中写：`throw new ExceptionName(p);` 这很像返回一个对象，但是返回对象将返回到上一个作用域，而这里的 `throw`，会将程序执行的位点抛出到很远的地方——异常处理程序。

抛出异常的异常对象必须是 Throwable 接口的，一般是 Exception 这个抽象类的导出类，又是也是 RuntimeException 的某种实例。

抛出异常可以携带参数，不过一般而言，仅仅依靠异常的类名就能猜出来异常的原因。

需要注意的是，抛出异常的方法体必须声明其可能抛出一个异常，否则，不能显式的使用 `throw` 以及任何编译器一眼就能看出的除了 Runtime.. 类型的异常。方法名称要这样写：`void methodA() throws ExceptionA, ExceptionB {}` 。只有这样才可以抛出异常，这叫做异常声明。

**监控区域、异常处理程序**

异常处理程序是被包裹在 `try {} catch {} finally {}` 语句块中的代码。所有可能出现的异常的代码呗放置在 try 之后的语句块中，而 catch 之后的语句块负责接管当 try 中的语句出现异常时的跳转，当 try 中某条语句出现异常，则其之后的语句不再执行，直接跳转到 catch 中。finally 中的语句一定会被执行，不论是否出错。

其中 try 后的区域称之为 监控区域。 catch 之后的区域称之为 异常处理程序。可以有多个异常处理程序 `try {} catch (ExceptionA a) {} catch (ExceptionB b) {} finally {}` 程序按顺序在 catch 中寻找对应类型，如果找到匹配，或者 catch 中的类型是发生类型的父类，那么之后的错误处理程序不再执行。

所有的异常用 `(ExceptionType e)` 这样包裹起来，其中 e 代表了如果发生异常，`ExceptionType e = ExceptionType(p)` e 接受 throw 抛出的对象，它是一个引用。在实际中，多调用 `e.printStackTrace()` 来向流打印堆栈错误信息。

**异常处理模式**

一般而言，早期的程序总是希望程序在出错的时候，通过异常处理程序能够解决这个问题，然后继续运行下去（恢复模式）。但这导致了异常处理和代码的强耦合。因此，现在的程序多是终止模式。当程序出现异常，只需要进行一些日志记录、控制台输出、像是文件或者网络的关闭等善后工作，因为 Java 带有自动的垃圾处理，因此 finally 作用并不是很大。此外，需要注意， try catch finally 三个语句块中的变量不在同一作用域中，也就是说，你在 try 中定义的一个变量，不能再 finally 中找到。

**异常的类型**

异常需要自己声明。一般而言，仅仅依靠异常名字提供的信息即可有效的工作。异常类必须继承自 Exception。可以在异常构造的时候，写入日志，可以为异常添加参数、详细信息，构建异常链等，在下一部分会着重介绍。

**RuntimeException**

这是唯一一个在抛出时不需要声明的异常，也是唯一一个在触发的时候不需要被包裹在 try 语句中的异常。因为这个异常默认指的是那些编译器无法发现的，在程序运行时出现的错误。如下的 `g()` 和 `main() 方法中调用 x.g()` 发生的事情。

下面一个例子介绍了 **异常的定义、异常的说明、异常的抛出、异常处理程序的捕获和处理、多个异常处理顺序、如何从异常中恢复**。

对于异常而言，捕获并且简单的调用 `e.printStackTrace()` 会向控制台 err 流输出信息，程序不会出错退出。

```java
import static com.mazhangjing.Print.*;
public class ExceptionDemo {
    //使用 throws 来说明可能发生的异常,Java会强制检查，如果发现异常而没有说明会去处理异常
    //则无法通过编译。可以在接口中预留可能发生的异常
    void f() throws GFException {
        print("I will throw a Exception now!\n");
        //使用 throw 抛出异常
        throw new GFException();
    }
    void f(String s) throws GFException {
        print("I will throw a Exception WITH String now!\n");
        throw new GFException(s);
    }
    void g() { //RuntimeException是唯一的特例，系统不会检查，也不用说明g()会发生此异常
        //其如果碰到，会自动的被printStackTrace()
        throw new RuntimeException();
    }
    public static void main(String[] args){
        ExceptionDemo x = new ExceptionDemo();
        //必须将f()放置在一个 try 的监控块中，否则程序就真的退出了
        try { x.f("No_GF Exception!"); } catch (GFException e) {
            //e.printStackTrace(); 类似于 traceback.print_exc() in python
            //不写的话，默认输出到错误流，写的话，可以输出到输出流
            e.printStackTrace(System.out);
        } finally {
            print("\nAre you handle the Exception? \nYes if you can see me");
        }
        //Runtime异常
        //x.g(); 自动被printStackTrace，不需要用户操心，指的是那些运行时发生的错误
    }
    public static class Ex {
        public static void main(String[] args){

            int[] a = {1,2,3};
            int n = 4;
            try {//捕获多个异常。注意，通常把Exception异常放在最下面
                A aa = null;
                aa.a();
            } catch (NullPointerException e) {
                e.printStackTrace(System.out);
            } catch (Exception e){
                e.printStackTrace(System.out);
            }
            //从异常中恢复示例
            while (true){
                try {
                    print(a[n]);
                    break;
                } catch (ArrayIndexOutOfBoundsException e) {
                    n--;
                }
            }
        }
    }
}
//简单的异常关键字即可
class GFException extends Exception {
    GFException() {}
    GFException(String s) {super(s);}
}
class A {
    void a() {}
}
```

# 自定义异常处理

这一节会介绍如何构造一个包含详细信息、能够进行日志记录的异常类，如何在异常处理程序中使用这个异常类的不同层面的信息。

```java
//高级错误处理技术：详细信息的添加和日志的记录、不同详细程度错误信息的获取
import com.mazhangjing.Print.*; import java.io.*; import java.util.logging.Logger;

public class Exception_Logging {
    static void f() throws NFException {
        throw new NFException();
    }
    static void f2() throws NFException {
        throw new NFException("f2() in E_L.class");
    }
    public static void main(String[] args){
        try{
            f2();
        } catch (NFException e) {
            print("MSG:" + e.getMessage()); //仅MESSAGE
            print("Cause:" + e.getCause()); //原因
            print("LocalMSG:" + e.getLocalizedMessage()); //一般同MESSAGE
            print("Trace:");
            e.printStackTrace(System.out); //详细错误位置
        }
    }
}
class NFException extends Exception {
    private static Logger logger = Logger.getLogger("NFException");
    public NFException() {
        this("");
    }
    public NFException(String detail){
        //可以使用getMessage()方法指定更加具体的错误信息。
        x = detail;
        Writer s = new StringWriter();
        printStackTrace(new PrintWriter(s));
        //不能这样写：直接转型会失败：(PrintWriter)s ClassCastException
        logger.severe(s.toString()); //s填充过内容后，toString发送到logger
    }
    private String x;
    public String getMessage(){
        //这里重载了父类的方法，需要注意，不要忘掉父类的方法也要写，否则会少很多信息
        return "Detail Message: " + x + super.getMessage();
    }
}

```

程序的 NFException 定义部分展示了如何在构造这个类的时候添加日志记录。如何为这个类提供详细的错误自定信息。

## 保存异常信息到日志

记录日志需要使用 `java.util.logging.Logger.getLogger()` 获取一个 Logger 对象。这个 Logger 一般保存这个错误类的名字信息。如同我们在私有方法中定义的那样。之后的问题就是，如何将 `printStackTrace` 的异常信息输送到 Logger， Logger 通过 `.severe("")` 来记录信息。因为 `printStackTrace` 只能将信息传递给 `printWriter`, 因此在这里我们需要一个 `printWriter` 对象保存信息。但这个对象不能 `toString`，因此将`printWriter` 中的信息发送给 `StringWriter`，然后导出信息。

其实这一步也可以使用 `getStackTracer` ，然后对 `StackTracerElement.toString()` 并且拼接成字符串，保存日志也可以。在下一节回介绍这些内容。

注意，这里为什么不直接转型 `printWriter` 到 `StringWriter`，而是要通过对象传递，是因为转型会失败。此外，`printWriter` 必须接受对象作为交互口，比如文件或者流或者另一个Writer。

## 为异常添加详细信息

重载 getMessage() 方法来讲构造时传入的信息返回出去。注意对于这里，之所以没有在没有参数构造器中写入日志，而是在其中调用了一个特殊的带有参数的构造器的原因是：当记录日志的时候，系统会访问父类的 getMessage 方法，但是，那个时候，这里的重载getMessage还未生效，因此需要这样写。

## 获取不同层次的异常信息

在异常处理程序中，如果每次都必须使用 `printStackTrace`，那么错误信息会有很多，因此，可以通过调用不同的 e 的方法来返回不同层次的异常信息，比如 `getMessage()` 返回详细信息， `getCause()` 返回异常链上一级，`getLocaizedMessage()` 获取本层信息等。可以根据需要选用。

# 异常的轨迹和层次

下列代码介绍了异常轨迹以及其获取的方法 `getStrackTrace、StrackTraceElement`。介绍了多层异常的不同需求的不同处理：清空上层信息并重新抛出的 `fillStrackTrace`、添加到异常链的 `initCause`等。

```java
import static com.mazhangjing.Print.print;
public class Exception_chain {
    public void f() throws NGException {
        throw new NGException();
    }
    public void g() throws Exception {
        try {
            f();
        } catch (Exception e) {//注意，这里要写Exception，不是NGException
            print("由g()中异常处理程序发现的异常：");
            e.printStackTrace(System.out);
            throw (Exception)e.fillInStackTrace();
            //这个重新抛出的异常仅说明其自身栈发生的异常
            //通过fill方法，会自动填充其异常名称，然后清空信息重新抛出
        }
    }
    public void h() throws NGException {
        try {
            f();
        } catch (Exception e) {
            NGException x = new NGException();
            x.initCause(e);//通过initCause来设置异常链
            throw x;
        }
    }
    public static void main(String[] args){
        //展示了使用 getStackTrace() 和 StackTraceElement 得到异常栈轨迹的方法
        Exception_chain x = new Exception_chain();
        try {
            x.f();
        } catch (Exception e) {
            print("In Exception process Field!");
            StackTraceElement[] s = e.getStackTrace();
            for (StackTraceElement i : s){
                print("=============LINE " + i.getLineNumber()
                        + "=============");
                print("ModelName: " + i.getModuleName());
                print("FileName: " + i.getFileName());
                print("ClassName: " + i.getClassName());
                print("AllinOne: " + i.toString());
            }
        }

        print("======== TEST RETHROWS EXCEPTION ========");
        //展示了重新抛出异常并保留异常名字但清除异常信息的方法
        try {
            x.g();
        } catch (Exception e) {
            print("由捕获g()的异常处理程序发现的异常：");
            e.printStackTrace(System.out);
        }

        print("======== TEST THROWCHANS EXCEPTION ========");
        //展示了异常链抛出异常的方法
        try {
            x.h();
        } catch (Exception e) {
            print("由捕获h()的异常处理程序发现的异常：");
            e.printStackTrace(System.out);
        }
    }
}
class NGException extends Exception {}
```

## 异常轨迹获取

异常在多级调用的过程中出现，使用 `printStackTrace` 会打印所有信息。而我们可以通过 `getStackTrace` 获取一个 `StackTraceElement` 的数组，这个数组包含了异出错的轨迹，比如下例中 main() 调用 g(), g() 调用 f()，而在 f() 中发生了异常，这就是一个异常轨迹。这个数组保存了这样的异常轨迹层次。对于这个数组中的每一个元素，都保存了一个轨迹位点的信息，其有 `getLineNumber, getModuleName, getFileName, getClassName, toString` 方法可供调用。下列代码调用异常位点方法的输出如下：

```java
=============LINE 6=============
ModelName: null
FileName: Exception_chain.java
ClassName: Exception_chain
AllinOne: Exception_chain.f(Exception_chain.java:6)
=============LINE 39=============
ModelName: null
FileName: Exception_chain.java
ClassName: Exception_chain
AllinOne: Exception_chain.main(Exception...
```

## 重复抛出异常

在 g() 中，捕获 f() 的 try 语句块可以捕获 f() 的异常，并且可以不进行处理并且重复抛出，只要 `throw e;` 即可。但是，有些时候，我们不想让 g() 的错误程序知道 f() 的存在和 f() 中发生的异常，因此可以选择 e.fillInStackTrace() 来生成一个新的 Throwable 的对象，然后将这个对象转型为 Exception 类型抛出。这样的话，我们新抛出的这个异常就有了和 f() 异常相同的名字，但是不包含 f() 异常的信息。这种优势是 `throw new Exception()` 所无法比拟的，因为后者不能指定异常名称，如果需要，必须写清类型，但这样又带来耦合的问题，因此使用 `e.fillInStackTrace()` 清空之前堆栈重新抛出更方便。

```java
======== TEST RETHROWS EXCEPTION ========
由g()中异常处理程序发现的异常：
NGException
	at Exception_chain.f(Exception_chain.java:6)
	at Exception_chain.g(Exception_chain.java:10)
	at Exception_chain.main(Exception_chain.java:56)
由捕获g()的异常处理程序发现的异常：
NGException
	at Exception_chain.g(Exception_chain.java:18)
	at Exception_chain.main(Exception_chain.java:56)
```
重复抛出异常的实例代码结果如上所示。

## 异常链

有些时候，我们需要来自上游的信息，这个时候，可以使用异常链。对于每一个异常，都内含了 `initCause(e)` 这样一个方法，可以将上一级的异常放入，然后返回这一级的异常，这样的话，通过 `e.getStackTrace()[n].StackTraceElement.getCause()` 即可获取异常链上游信息。 

```java
======== TEST THROWCHANS EXCEPTION ========
由捕获h()的异常处理程序发现的异常：
NGException
	at Exception_chain.h(Exception_chain.java:29)
	at Exception_chain.main(Exception_chain.java:65)
Caused by: NGException
	at Exception_chain.f(Exception_chain.java:6)
	at Exception_chain.h(Exception_chain.java:27)
	... 1 more
```

上述示例代码执行结果如下。

## 如何选择层次异常处理方式？

换句话说，现在我们有多层的异常需要处理。那么，如果希望屏蔽上层异常并且重新制定类型，使用 `throw new Exception()` 即可。如果希望屏蔽上层异常但是返回相同异常类型，使用重新抛出 `throw (Exception)e.fillStrackTrace()` 即可。 如果希望包含上层信息的话，使用 `Exception x = new Exception(); x.initCause(e); throw x;` 异常链即可。

# 异常的匹配

默认的，匹配是按照代码顺序进行的，当其发现一个声明是目前错误的类或者父类的时候，就默认进入，而忽略其它的。这就是异常的匹配思路，可以看到，下列代码中的异常继承结构下，将 C1 转型为 A1 之后，执行 getExc() 动态绑定，这时的异常应该是 Exc3。而其进入了基类 Exc 的处理程序。

```java
import static com.mazhangjing.Print.print;

public class Match {
    public static void main(String[] args){
        A1 x = new C1();
        try {x.getExc();} catch (Exc e) {print(e.toString());} //Exc3
    }
}
class A1 {
    void getExc() throws Exc {
        throw new Exc();
    }
}
class B1 extends A1 {
    void getExc() throws Exc {
        throw new Exc2();
    }
}
class C1 extends B1 {
    void getExc() throws Exc {
        throw new Exc3();
    }
}
class Exc extends Exception {}
class Exc2 extends Exc {}
class Exc3 extends Exc2 {}
```

# 异常的限制

异常可以声明在 接口、 抽象类 中，这样继承或者实现这些标准的类则必须按照某种规则去处理这些异常。其中接口和类继承的处理方式并不相同，声明在构造器中的异常和声明在方法中的异常也不相同。

```java
public class Limits extends Inning implements Strom{
    Limits() throws BaseExp{
        //和方法重载不用重新声明不同，编译器要求这里至少必须实现基类的声明
        throw new Strike(); //可以抛出任何继承自基类的异常
        //throw new RainOut(); //不可以，如果不是基类的异常，则需要声明

    }
    public void atBeat() {//重载的方法可以处理基类中所定义的异常
    }

    public void rainHard() throws RainOut{
        throw new RRainOut();
    }
    //public void walk() throws BaseExp {}
    //如果基类没有声明异常，那么所有继承方法不得声明异常，因为这样做
    //通过重载的话，在父类无法进行捕获
    public void walk() {
        super.walk();
    } 
    //如果没有重载的话，则可以声明异常
    public void belong_son() throws RainOut {}
    
    public void event(){ }
    //注意，这里的 event() 不能声明 BaseExp，因为它也是接口的实现，所以必须和接口保持一致
    //这里的 event() 同样不能声明 Rainner，因为它可能向上转型，而基类没有声明 Rainner
}
class BaseExp extends Exception {}
class Foul extends BaseExp {}
class Strike extends Foul {}
class RainOut extends Exception {}
class RRainOut extends RainOut {}

/**抽象基类 Inning 对于构造器、非abstract方法、abstract方法的异常声明*/
abstract class Inning {
    //在标准中添加异常声明可以强制用户在其实现中捕获可能出现的异常
    public Inning() throws BaseExp {}
    public void event() throws BaseExp {}
    public abstract void atBeat() throws Strike, Foul;
    public void walk() {}
}

/**接口 Strom 每个类定义都包含了自身的异常声明*/
interface Strom {
    void rainHard() throws RainOut;
    void event() throws RainOut;
}
```

注意这里的 event(), 它非常惨，因为是继承又是实现，继承允许其声明基类异常，但是实现不允许出现没有定义的异常，所以这里面不能处理任何异常。

以上代码简要说明了异常在类和接口中由于实现带来的限制。具体而言：

- 对于**接口**而言,**可以不写，不能多写**，规则很简单，所有的接口方法都必须强制通过接口实现得到引用，因此其可以不写声明，但是不能写接口定义没有定义的声明。这里的可以不写不是指不写就可以抛出声明，而是说，可以不写并不抛出声明。换句话说，**接口的实现比接口的定义权限窄。**


- 对于**继承**而言，规则稍微麻烦一点，因为不确定是否通过基类还是在本地调用方法。


    - 对于**重载方法**而言,**可以不写，不能多写**，由于不确定在本地或者基类调用，因此可以不写，但是不能抛出异常。可以声明基类出现过的声明，但是不能声明基类没有实现的声明，因为不确定是否会通过父类。换句话说，**子类的权限比父类窄。**

    - 对于**非重载方法**而言,**爱写不写**，要抛出什么就实现什么。


- 对于**构造器**而言,**不能不写，可以多写**，必须强制实现基类构造器的声明，可以使用任意继承自此声明额异常而不用进一步声明，不能声明任何基类不存在的异常，因为肯定要调用父类构造器。


- 对于任何的标准，只要能处理某个异常，那么一定可以处理继承自这个异常的所有异常。

# 异常的细节

## 构造器的异常处理

如果在方法中出现异常，那么使用 try catch finally 可以很好的完成任务，但如果在构造器中出现异常，就比较麻烦。下面例子的构造器接受文件路径，并试图打开文件。有趣的是，我们希望在构造器创建对象后文件不关闭，而如果打开文件，但是读取错误，则希望在构造器中就关闭文件。为了实现这一需求，我们将关闭我文件放在 catch 中，而不是 finally 中。此外，关闭文件会出错，因此在 catch 中还要嵌套一个 try catch 语句。

注意，这里提供了两种使用这个 InputFile 类的例子，其一包含了完整的异常处理，其二则完全放弃了处理，可以看到，后者更间接，但是问题更多，比如，如果打开出错，那么这个文件就永远得不到清理，因此需要使用一个专门的清理类去做善后工作。而对于完全处理，只需要编写这个方法，然后将文件名传递给它就行，这个方法注定是不会出错的，因为编译器检查过，对于任何可能的异常都进行了处理并且没有触发新的异常。

```java
import java.io.*; import static com.mazhangjing.Print.*;
public class InputFile {
    private BufferedReader in;
    InputFile(String filename) throws Exception {
        try {
            in = new BufferedReader(new FileReader(filename));
        } catch (FileNotFoundException e) {//如果找不到文件，不用关闭
            //这也是为什么不能写在 finally 中的原因
            System.out.println("Can't open file " + filename +" "+ e);
            throw e;
        } catch (Exception e) {
            try {
                in.close();//因为关闭文件也有可能存在异常
            } catch (IOException ei){
                System.out.println("Close File "+filename+" Failed.");
            } throw e;
        } //finally {}
        //因为文件在整个对象存活期间一直存在，因此不要在这里关闭，否则调用完毕构造器文件就关闭了
    }
    public String getLine() {
        String s;
        try {
            s = in.readLine();
        } catch (IOException e) {
            throw new RuntimeException("readLine() failed");
        }
        return s;
    }
    public void dispose() {
        try {
            in.close();
        } catch (IOException e) {
            throw new RuntimeException("in.close() failed");
        }
    }
    public static void main2(String[] args){
        //一个绝对不会抛出可能的错误的示例程序
        InputFile file;
        try {
            file = new InputFile("C:/rookery.sql");
            try {
                for (int i = 0; i < 10; i++){
                    print(file.getLine());
                }
            } catch (Exception e){
            } finally {
                file.dispose();
            }
        } catch (Exception e){ //对于一个可能出现错误的类，必须进行处理，否则无法通过编译
        } finally {
            //因为try和finally不在一个作用域中，因此，不能在这里关闭文件
            //file.dispose(); 编译器错误，可能未初始化file
        }
    }
    public static void main(String[] args) throws Exception {
        //另一种写法是这样，如果声明了这个方法可以抛出错误，那么内部就会很简洁
        //之后在外面套一个壳子捕获所有种类的错误即可，从输出流中查看错误信息
        InputFile file = new InputFile("C:/rookery.sql");
        for (int i = 0; i < 10; i++) {
            print(file.getLine());
        }
        file.dispose();
        //但是吧，如果读取到一半出错，这个文件就永远不会被关闭了....
    }
}
```

## 声明还是处理异常？

需要注意，应该在某个地方提供异常的完全处理而不是声明一堆可能出现的异常，因为在上面看到了，继承和接口对于异常的限制很多。因此，在合适的方法中捕获和处理所有异常并创建新的异常会极大的提高接口和继承对于异常的容忍程度。对于C而言，他们倾向于不处理任何异常，根据编译器跟踪位置去修正错误。而Java提供了这种完全处理异常的能力，因此，在合适的实际将异常包装并完全处理而不是继续声明能简化由于异常过多造成的限制。


## 再看 Finally 

Finally 对于那些没有垃圾回收机制的语言来说很重要，但是Java自带了垃圾回收，因此，在这里主要的作用是关闭文件、网络连接、进行一些事后工作。Finally 在之前的Java版本中很容易出错，比如你在 finally 返回一个异常，return 一下，都会造成信息丢失。但是现版本已经修复了这个问题（Java SE10）。

```java
import static com.mazhangjing.Print.print;
public class Finally {
    static void f() throws MEx {
        throw new MEx();
    }
    static void g(){
        throw new RuntimeException();
        //throw new SMx();
    }
    public static void main(String[] args){
        try{
            Finally.f();
        } catch (Exception e){
            print(e);
        } finally {
            //return; //现在的Java版本不会在 return 后丢失之前错误信息
            Finally.g(); //现在的Java不允许在 finally 中出现未经handle的错误
            //也就是说，如果一个方法声称其可能出现错误，则不能放置在这里
            //即便出现错误，也可以捕获 try 语句内部的错误，不会造成信息丢失
        }
    }
}
class MEx extends Exception {
    public String toString() {
        return "MEx handing...";}
}
class SMx extends Exception {
    public String toString() {
        return "SMx handing...";}
}
```

在之前的Java中， finally 中如果出现 return 或者 finally 中的语句出现异常，则之前 try 中的异常会消失，现在编译器强制检查 finally 中的语句，不允许出现任何声明了异常却没有捕获的情况。如果真的出现异常，比如 `RuntimeException`,那么在 `Try` 中的异常不会消失。

## 活用 RuntimeException

在刚开始的时候，由于程序结构尚未搭建起来，因此，过早的创建一堆异常会造成很大的混乱，尤其是 Java 强制对异常的声明，可以预见的，这种行为造成的结果就是——所有的声明都是 `throws Exception`。所有的 catch 都是 `catch (Exception e) { pass };`。因此，可以在出现异常时，将异常栈打印出来，或者使用 `RuntimeException(e)` 来报错，这样可以提醒自己这里可能出错，不至于最后忘记处理错误。

```java
//Tips:对于没有定义的异常，catch后调用printStackTrace，或者使用这样：
try{
    x.getExc();
} catch (Exception e) {
    throw new RuntimeException(e);
}//这样也可以避免吞并错误，并且能够真的报错，而如果仅仅打印，则程序还会继续运行
//不论如何，这两种方式能够帮助在暂时不知道如何处理异常的时候不至于忘记它
```